# Installing the Requirements

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00


# Importing the libraries

In [ ]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import time

## Check the type of device we are using

In [ ]:
# Check if CUDA (GPU) is available
use_cuda = torch.cuda.is_available()

# Set the device to either CUDA (GPU) or CPU based on availability
device = torch.device("cuda" if use_cuda else "cpu")

# Print information about the selected device
if use_cuda:
    print("Using CUDA (GPU)")
else:
  print("Using CPU")

Using CPU


# Loading the base model over here

In [ ]:
# Load the base RoBERTa model
base_model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

# Providing the input for checking the base model latency

In [ ]:
# Define your input text and process it with the tokenizer
input_text = "It's been a long day without you my friend"
input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt")

# Fetching the latency of base model

In [ ]:
# Measure the initial model's latency
start_time = time.time()
_ = base_model(input_ids)
initial_latency = time.time() - start_time

# Calculating the latency of the base model

In [ ]:
# Print initial metrics (latency, GPU memory)
print(f"Initial Latency: {initial_latency} seconds")

Initial Latency: 0.24855589866638184 seconds


# Converting the base model into quantized model

In [ ]:
import torch
from transformers import RobertaForSequenceClassification

# Quantize the model
quantized_model = torch.quantization.quantize_dynamic(base_model, {torch.nn.Linear}, dtype=torch.float16)



In [ ]:
quantized_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.float16)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.float16)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.float16)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
           

# Providing the input for checking the quantized model latency

In [ ]:
# Define your input text and process it with the tokenizer (as shown previously)
input_text = "It's been a long day without you my friend"
input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt")

# Fetching the latency of quantized model

In [ ]:
# # Warm-up the model (optional but recommended)
# with torch.no_grad():
#     _ = quantized_model(input_ids)

In [ ]:
# Measure the quantized model's latency for a single inference
start_time = time.time()
with torch.no_grad():
    _ = quantized_model(input_ids)
latency = time.time() - start_time

# Calculating the latency of the quantized model

In [ ]:
print(f"Latency of Quantized Model for Single Inference: {latency} seconds")

Latency of Quantized Model for Single Inference: 0.08593893051147461 seconds


# Comparing the parameters of base and quantized model

In [ ]:
# Get the model sizes (number of parameters) for both base and quantized models
base_model_size = sum(p.numel() for p in base_model.parameters())
quantized_model_size = sum(p.numel() for p in quantized_model.parameters())

# Print the model sizes
print(f"Number of Parameters in Base Model: {base_model_size}")
print(f"Number of Parameters in Quantized Model: {quantized_model_size}")

Number of Parameters in Base Model: 124648708
Number of Parameters in Quantized Model: 39037440


## Text Classification Using the base model over here

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)



In [ ]:
text = "I love risk"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Celebrating my promotion 😎"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) joy 0.7815
2) optimism 0.1173
3) sadness 0.0546
4) anger 0.0466


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

## Text Classification Using the quantised model over here

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='emotion'
# MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)



In [ ]:
text = "manmeet singh is the best person in colloquial"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = quantized_model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Celebrating my promotion 😎"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) joy 0.8767
2) optimism 0.0624
3) anger 0.043
4) sadness 0.0178


In [ ]:
quantized_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.float16)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.float16)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.float16)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
           